# How to use PyARPES (private version)

## Introduction

There is a python library called [pyarpes](https://arpes.readthedocs.io/) for analyzing ARPES data.
It seems to be quite incomplete and has some buggy functions. However, it is still usable (or so I think).
I was wondering what is the difference between this and Igor's analysis?  I will end up using Igor, but it might be useful when handling a large amount of data.
Basically, it is designed to handle files that extend xarray, and xarray itself seems to be stable, so its reliability is relatively high.

Unfortunatelyy, this package is no longer maintained very well, and in many ways it is not a modern Pythonic description. I decided to manage it myself. Let's aim to make it a robust package.


## autoreload related

Eventually, following three lines are not essential.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## import 

Let's do the following:

* Well, we need numpy in any case.
* It's better to set matplotlib explicitly to easily tune the graph.
* For coloring the ARPES data by logarithmic scale, we introduce the LogNorm.
* arpes.config is (probably) required absolutely
* ~~pes.SPD_main.py is written by me, that why the the directory is not arpes~~ Now, SPD_main.py is moved to my pyarpes repository. 
* qt_tool is used to display the data by using PyQt.
* ktool is the same as above but for momentum analysis?
* convert_to_kspace is used to convert from angle to momentum. 
* hvplot.xarray may not be needed.  If you want to use Bokeh.

Note that you need enough libraries installed to load the above libraries and not get an error.  However, on the other hand, libralies in pyproject seems to be sufficient for usual task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import hvplot.xarray

import arpes.config
from arpes.io import load_data
from arpes.utilities.conversion.core import convert_to_kspace

## Loding　Data

In order to load the data you've measured yourself, it is essential to use:

* from arpes.io  import load_data 



If you feel uncertain about the contents of your data, you can use:

* from arpes.io import example_data



```python:
ex = example_data.cut
```

By examining the contents of ex and comparing them, you might find some hints.


In [ ]:
from arpes.io import example_data

The function `load_data` is easy to use for just importing as follows:

In [ ]:
f = load_data(
    "/Users/arafune/src/arpes/src/arpes/example_data/example_itx_data.itx",
    location="SPD_phoibos"
)
# f2 = load_data("/Users/arafune/src/analysis_tool/test/pes/GrIr.sp2", location="SPD_phoibos")

However, if you add another information that was not added in `User Comments` such as:

* the value of x, y, z -axis, or temperature,

You can use:

```python
f=load_data("/Users/arafune/src/analysis_tool/test/pes/PES_16_Spectrum_3.itx",location="SPD_phoibos",
             x=1,
             sample_name="Cu(111)")
```

Variable names like ''x'' and ''sample_name'' are recommended to be used as they are pre-defined. 
Otherwise, they would just remain as regular comments. 
There seems to be no list or similar thing indicating what variable names are official, so you might need to check after actually loading the data.

Both `sample_name` and `sample_workfunction` are variable names that require some understanding. 
They are not simply "sample" or "workfunction" (the latter is reserved for the analyzer's work function). 
This manual doesn't provide additional information, at least not in this section.

You'll have to refer to the `xarray_extensions.py` for details.

Following outputs may be hinted as the "standard" variable name.


In [ ]:
f.S.polarization

f.attrs
```python
{'spectrum_type': 'cut',
 'Created Date (UTC)': '2022-Feb-28 05:03:33.001919',
 'Igor Text File Exporter Version': 0.3,
 'Created by': 'SpecsLab Prodigy, Version 4.86.2-r103043',
 'Scan Mode': 'Fixed Analyzer Transmission',
 'User Comment': '',
 'Analysis Mode': 'UPS',
 'Lens Voltage': '40V',
 'Analyzer Slits': '1:0.5x20\\B:open',
 'Number of Scans': 30,
 'Number of Samples': 501,
 'Scan Step': 0.002,
 'DwellTime': 0.096,
 'Kinetic Energy': 9,
 'Binding Energy': -4.008,
 'Bias Voltage': -1.5,
 'angle_unit': 'rad (theta_y)',
 'enegy_unit': 'eV (Kinetic Energy)',
 'count_unit': 'cps (Intensity)',
 'trace': Trace(silent=False, start_time=1688363879143870000),
 'hv': 4.992,
 'workfunction': 4.401,
 'lens_mode': 'WideAngleMode',
 'pass_energy': 5,
 'mcp_voltage': 1500,
 'id': 2,
 'x': nan,
 'y': nan,
 'z': nan,
 'theta': 0,
 'beta': 0,
 'chi': 0,
 'alpha': 1.5707963267948966,
 'energy_notation': 'Kinetic',
 'analyzer': 'Specs PHOIBOS 100',
 'analyzer_name': 'Specs PHOIBOS 100',
 'parallel_deflectors': False,
 'perpendicular_deflectors': False,
 'analyzer_radius': 100,
 'analyzer_type': 'hemispherical',
 'psi': 0,
 'chi_offset': 0}
 ```

f.S.spectrometer_settings
```python
{'lens_mode': 'WideAngleMode', 'pass_energy': 5}
```

f.S.experimental_conditions

```python
{'hv': 4.992, 'polarization': None, 'temp': None}
```

f.S.scan_info
```python
{'time': None,
 'date': None,
 'type': None,
 'spectrum_type': 'cut',
 'experimenter': None,
 'sample': None}
```

f.S.sample_angles
```python
(0,
 0,
 0,
 <xarray.DataArray 'phi' (phi: 600)>
 array([-0.217803, -0.217076, -0.216349, ...,  0.216349,  0.217076,  0.217803])
 Coordinates:
   * phi      (phi) float64 -0.2178 -0.2171 -0.2163 ... 0.2163 0.2171 0.2178
     x        float64 nan
     y        float64 nan
     z        float64 nan
     theta    int64 0
     beta     int64 0
     chi      int64 0
     hv       float64 4.992
     alpha    float64 1.571
     psi      int64 0,
 0,
 1.5707963267948966)
```


f.S.sample_pos
```python
(nan, nan, nan)
```
x, y, z

f.S.experiment_info
```python
{'temperature': None,
 'temperature_cryotip': None,
 'pressure': None,
 'polarization': (None, None),
 'photon_flux': None,
 'photocurrent': None,
 'probe': None,
 'probe_detail': None,
 'analyzer': 'Specs PHOIBOS 100',
 'analyzer_detail': {'name': 'Specs PHOIBOS 100',
  'parallel_deflectors': False,
  'perpendicular_deflectors': False,
  'type': 'hemispherical',
  'radius': 100}}
```


f.S.laser_info
```python
{'probe_wavelength': None,
 'probe_energy': 4.992,
 'probe_fluence': None,
 'probe_pulse_energy': None,
 'probe_spot_size': (None, None),
 'probe_profile': None,
 'probe_linewidth': None,
 'probe_temporal_width': None,
 'probe_polarization': (None, None),
 'pump_wavelength': None,
 'pump_energy': None,
 'pump_fluence': None,
 'pump_pulse_energy': None,
 'pump_spot_size': (None, None),
 'pump_profile': None,
 'pump_linewidth': None,
 'pump_temporal_width': None,
 'pump_polarization': (None, None),
 'repetition_rate': None}
```

f.S.probe_info
```python
{'probe_wavelength': None,
 'probe_energy': 4.992,
 'probe_fluence': None,
 'probe_pulse_energy': None,
 'probe_spot_size': (None, None),
 'probe_profile': None,
 'probe_linewidth': None,
 'probe_temporal_width': None,
 'probe_polarization': (None, None)}
```

f.S.pump_info
```python
{'pump_wavelength': None,
 'pump_energy': None,
 'pump_fluence': None,
 'pump_pulse_energy': None,
 'pump_spot_size': (None, None),
 'pump_profile': None,
 'pump_linewidth': None,
 'pump_temporal_width': None,
 'pump_polarization': (None, None)}
```

f.S.analyzer_info
```python
{'lens_mode': 'WideAngleMode',
 'lens_mode_name': None,
 'acquisition_mode': None,
 'pass_energy': 5,
 'slit_shape': None,
 'slit_width': None,
 'slit_number': None,
 'lens_table': None,
 'analyzer_type': 'hemispherical',
 'mcp_voltage': 1500,
 'work_function': 4.401}
```

## Plot

The functionality is simple. 
Regarding this aspect, it's safer to consider it as a feature of xarray rather than pyarpes. 
As mentioned earlier, xarray seems to be relatively well-maintained. 

### matplotlib base: S.plot()

In [ ]:
f.S.plot()

By following, you can use the functionality of matplotlib.pyplot directly.

In [ ]:
# However, the plot method of xarray is merely a thin wrapper around matplotlib. Therefore, if you want to change the output format, it's better to do so as follows:
fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(111)
f.S.spectrum.T.plot(ax=ax)

The plot method of xarray is merely a thin wrapper around matplotlib. 
Therefore, if you want to change the output format, it's better to do so as follows:

In [ ]:
f.S.spectrum.T.plot(aspect=1.3, size=6, cmap=plt.cm.plasma)
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

In [ ]:
# Logarithmic scale
f.S.spectrum.T.plot(aspect=1.7, size=4, cmap=plt.cm.jet, norm=LogNorm(vmin=1, vmax=100))
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

### Matplotlib base: stack-trace

In [ ]:
from arpes.plotting import stack_plot

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=20,
    scale_factor=0.3,
    title="2PPE Xe/Au(111)",
    linewidth=0.3,
    color="plasma",
    shift=0.00,
    mode="hide_line",
    label="label test",
    figsize=(7, 5),
)

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=10,
    scale_factor=0.1,
    title="2PPE Xe/Au(111)",
    linewidth=0.5,
    color="plasma",
    shift=0.00,
    label="label test",
    mode="fill_between",
    figsize=(7, 5),
    offset_correction="zero",
)

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=130,
    title="2PPE Xe/Au(111)",
    linewidth=0.5,
    color="viridis",
    shift=0.00,
    label="label test",
    mode="hide_line",
    figsize=(7, 4),
)

#### Bokeh base 

You can also achieve interactive visualization using Bokeh. While it should be possible to use Bokeh directly, I'll try using hvplot here. Note that there is no codes about hvplot in pyarpes .

~~However, please note that the output from hvplot might not be viewable in VS Code.~~


In [ ]:
f.hvplot.image(colorbar=True)

In [ ]:
f.S.spectrum.T.hvplot(invert=True, frame_height=200, frame_width=400)

## Conversion into k-space

There is a straightforward function called `convert_to_kspace`. In the following example, kp is specified, but if not provided, it automatically determines the appropriate range. ~~However, the treatment of the energy axis needs to be verified.~~ Currently, the data records whether it has the energy axis as Kinetic Energy (Final State Energy) or as Binding Energy. Based on this information, the function automatically and appropriately performs θ-k conversion.

In [ ]:
convert_to_kspace(f.spectrum, kp=np.linspace(-0.25, 0.25, 100)).plot()
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("$k_{//}$")

## Derivative

## Combination

### Concatenate along angle

### Merge time resolved data into single DataSet

## Misc


### calculate_background_hull


In [ ]:
from arpes.analysis.background import calculate_background_hull

**NOTE**:
This function can be applied to 1D spectrum data.
This means that it cannot be use for ARPES data.*

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
calculate_background_hull(f.spectrum.sum("phi")).S.plot(ax=ax)
f.spectrum.sum("phi").S.plot(ax=ax)

### local_minima


In [ ]:
from arpes.analysis.xps import local_minima

In [ ]:
eV = f.spectrum.sum("phi").coords["eV"].values
intensity = f.spectrum.sum("phi").values